# Microsoft Entra ID for Azure SQL Databases

## Motivation

As organizations increasingly embrace cloud-based solutions, the need for secure and seamless access management becomes paramount. Microsoft Entra ID is Microsoft's cloud-based identity and access management service, designed to help organizations manage and secure their users, applications, and data. When it comes to Azure resources, including Azure SQL Database, traditional username/password-based authentication may not provide the level of security and convenience required by modern cloud-based applications. That's where Microsoft Entra ID integration comes in.

Integrating Microsoft Entra ID with Azure SQL Database offers a range of advantages that enhance security, simplify user management, and enable a more seamless user experience. This integration leverages the power of identity-based authentication and access control, providing a robust and scalable solution for managing database access in a multi-tenant cloud environment.

## Benefits of integrating Microsoft Entra ID with Azure SQL Database

Integrating Microsoft Entra ID with Azure SQL Database offers several compelling benefits:

- **Enhanced Security**: With Microsoft Entra ID's robust security features, such as **Multi-factor Authentication (MFA)**, you can fortify your Azure SQL Database against unauthorized access and potential security threats

- **Simplified User Management**: By leveraging Microsoft Entra ID for user authentication and authorization, you can centralize user management, making it easier to add, remove, and update user access across your entire Azure ecosystem

- **Seamless Application Integration**: Microsoft Entra ID uses industry-standard protocols, enabling seamless integration with various applications and services. This means you can extend Microsoft Entra ID authentication to your custom applications or third-party services.

## Microsoft Entra admin for Azure SQL Database

> The Microsoft Entra admin is a critical entity when configuring Microsoft Entra ID authentication for Azure SQL Database. The Microsoft Entra admin is responsible for managing access to the database and its resources. It can be an Microsoft Entra user or a security group defined within the Microsoft Entra ID directory.

As the Microsoft Entra admin, this entity has elevated permissions over the Azure SQL Database, such as the ability to create and manage users, assign roles, and control access to the database. It is crucial to ensure that the Microsoft Entra admin is a trusted and responsible user or group within your organization, as they play a central role in the security and management of the database.

### Hands-On: Creating an Azure SQL Database Admin Account

Before continuing with this hands-on, first make sure you have followed the necessary hands-on from the previous lessons in this pathway. If you had, you should have a production Azure SQL Database. This database is the restored database after mimicking data loss in the original production database, which in the previous examples was called `production-database-restored`. The database should also be the primary database not the geo-replica that is hosted on a different SQL Server.

If you have this database set up as mentioned above, you can follow the next steps to configure Microsoft Entra ID access to this database:

- Navigate to the Azure portal and open the SQL Server that hosts your primary database `production-database-restored` (or your equivalent database)

- Under the **Security** section, click on **Microsoft Entra**, this should redirect you to the following page:

<p align="center">
    <img src="images/ServerAD2.png" height="400" width="950"/>
</p>

- Click on **Set admin** to assign an Microsoft Entra user or group as the Microsoft Entra admin for the SQL Server. This admin will have elevated privileges to manage the server and its databases. In this example I will go to **Users** and select my user as the admin of the database.

<p align="center">
    <img src="images/SelectAdmin2.png" height="500" width="850"/>
</p>

- Click **Select** to continue and **Save** to save the changes to your SQL Server. Now the user `Maya` will the Microsoft Entra admin of this SQL Server. 

Let's test this by connecting to the primary database using Azure Data Studio. First if you already have established a connection to this database in Azure Data Studio you can right-click on the server name and click **Disconnect**.

<p align="center">
    <img src="images/DisconnectServer.png" height="550" width="900"/>
</p>

Then double-click on the server name to connect to this primary database again. This time instead of using **SQL Login** as the authentication type we will use **Azure Active Directory - Universal with MFA support**. Note here, that Microsoft Entra ID used to be called Azure Active Directory before and Azure Data Studio hasn't been updated yet. This might change in the future, so the field mentioned above might become something along the lines **Microsoft Entra ID- Universal with MFA support**.

Next we will need to add account under the **Account** field. To do so click the **Add an account..** button from the drop-down menu. This will redirect you to the login page for Azure. Once you have successfully logged in your **Connection** pane should look like this:

<p align="center">
    <img src="images/ADConnect.png" height="750" width="800"/>
</p>

Click the **Connect** button to establish the connection to your Azure SQL Database. Once connected, you can explore the database, run queries, and manage the database objects using Azure Data Studio. As the admin account, you will have the necessary privileges to perform administrative tasks on the database.

## Microsoft Entra Users for Azure SQL Database

In addition to the admin account, you can grant access to Azure SQL Database for specific Microsoft Entra users or groups. Microsoft Entra users can have varying levels of permissions, depending on their roles and responsibilities within your organization. 

Common Roles for Microsoft Entra Users in Azure SQL Database include:

- `db_datareader`: This role grants read-only access to the database, allowing users to view data but not modify or alter it

- `db_datawriter`: This role grants write access to the database, allowing users to modify data but not perform certain administrative tasks

- `db_owner`: This role provides full control over the database, including the ability to modify schema, data, and grant permissions to other users

### Hands-On: Creating a DB Reader User

Let's walk through the process of creating a DB Reader user with read-only access to the Azure SQL Database:

- Navigate to the Azure portal and navigate to the Microsoft Entra ID homepage. Here create a new user that will ultimately be assigned the `db_datareader` role. Select a suggestive name for the user, such as **DB_Reader**. Create a password for this user using the **Let me create the password** field.

- Open Azure Data Studio and connect to the desired database (in this case the primary database) using the Microsoft Entra Admin credentials, just like you did before in the previous section. Right-click on the server connection and select **New Query** to open the query editor.

- In the query editor, run the following SQL query to grant the `db_datareader` role to the **DB_Reader** user:

In [ ]:
CREATE USER [DB_Reader@yourdomain.com] FROM EXTERNAL PROVIDER;
ALTER ROLE db_datareader ADD MEMBER [DB_Reader@yourdomain.com];


Where `[DB_Reader]` represents the name of the Microsoft Entra user you just created, in this example **DB_Reader**. You can find the name of your domain from the Microsoft Entra Directory user page, under **User principal name**. The domain it's what comes after @.

<p align="center">
    <img src="images/Domain.png" height="550" width="900"/>
</p>

### Hands-On: Test the DB Reader's User Access

To test the new user's connection:

- In Azure Data Studio re-connect to the Azure SQL Server you are currently connected to using the new user credentials. You can do this by right-clicking on the server name and selecting **Edit Connection**.

- This will open the **Connection** page. Here under **Account** click **Add account..** to add the new account. Follow through the log in process. Use the principal name from above as the email address to log in with.

- On the first log in you will be asked to set up a new password. Once the account has been added you are ready to press **Connect** to connect to the database using the newly created user.

- With the DB Reader user, you should now have read-only access to the database and be able to view its data. For example, if you right-click on the `dbo.dim_card_details` table and select **Select Top 1000** you should see the following output:

<p align="center">
    <img src="images/Top1000.png" height="500" width="900"/>
</p>

- But if you know try to run a query to delete the first entry in the `dbo.dim_card_details` table: `DELETE TOP (1) FROM dbo.dim_card_details` you should see the following error:

<p align="center">
    <img src="images/Error.png" height="450" width="900"/>
</p>

Indicating the user doesn't have access to modify the data, but read-only access as expected.

## Debugging Authentication and Connectivity Problems

When implementing Microsoft Entra ID authentication with Azure SQL Database, you might encounter various issues related to authentication and connectivity. Let's explore some common problems and how to troubleshoot and resolve them effectively.

### Debugging Common Authentication Issues

- **1. Invalid Credentials or Permissions**: Verify that the Microsoft Entra user or application identity used in the connection string has the appropriate permissions (e.g., `db_datareader`, `db_datawriter`) assigned to the target database

- **2. Missing Azure AD Registration**: Ensure that your application is properly registered in Microsoft Entra ID, and you have obtained the correct client ID and tenant ID to use in the authentication process. Again, to keep in mind here the error message might show up as missing Azure AD registration, Azure AD is simply the old nomenclature for Microsoft Entra ID, and Azure Data Studio have not updates the naming as of now.

- **3. Expired or Revoked Tokens**: Check for token expiration or token revocation issues. Tokens used for authentication have a limited validity period, and expired or revoked tokens will result in authentication failures. 

> In Azure Data Studio authentication tokens are only available for 12 hours. To obtain a new authentication token you will need to remove the Azure account you are logged in with and log in again. Alternatively you can remove the server connection and add a new connection to refresh the token.

## Key Takeaways

- Integrating Microsoft Entra ID with Azure SQL Database provides a more secure and manageable way to authenticate users and applications, eliminating the need for storing database credentials
- The Microsoft Entra admin is a highly privileged user who has full control over the Azure SQL Database resource. They can manage access control, configure firewall rules, and perform administrative tasks.
- Microsoft Entra users can have various roles, such as `db_datareader`, `db_datawriter`, `db_owner`, etc., which define their permissions and access levels within the database
- Common authentication issues may include invalid credentials, incorrect connection strings, missing Microsoft Entra ID registration or expired/revoked tokens. Debug and verify settings to resolve authentication problems.